In [1]:



import torch 



what we can quantize ? 
1- the wieghts
2- the activation 

after quantization we get : 
- samller models  
- speed gain 
- faster operation 

quantization after the training called POST-TRAINING QUANTIZATION 


## linear quantization is a popluar method of quantization 

in linear quantization we map high presision value to lower
like fp32 to int 8 

Formula : 

r = s(q - z)

where : 
s : scale (as the original tensor)
z : zero point (the quantized tensor)

getting q : 

form the original formaula above 

r/s = q - z 

q = r/s + z 

q = round(r/s + z)
q = int (round(r/s + z))



In [3]:
def linear_Q(
        tensor,
        scale,
        zero,
        dtype = torch.int8
):
    
    scaled_shifted_tensor = tensor / scale + zero 
    rounded = torch.round(scaled_shifted_tensor)

    q_min = torch.iinfo(dtype).min 
    q_max = torch.iinfo(dtype).max 

    q_tensor = rounded.clamp(q_min, q_max).to(dtype)

    return q_tensor

In [6]:
test_tensor = torch.tensor(
    [[191.5 , -13.5, 728.6],
     [92.14, 295.5, -184],
     [0 , 684.6 , 245.5]
    ]
)



In [7]:
scale = 3.5 
zero = -70 

quantized_tensor = linear_Q(test_tensor,scale,zero)

print(quantized_tensor)

tensor([[ -15,  -74,  127],
        [ -44,   14, -123],
        [ -70,  126,    0]], dtype=torch.int8)


In [8]:
dequant_tensor = scale * (quantized_tensor.float() - zero)
dequant_tensor

tensor([[ 192.5000,  -14.0000,  689.5000],
        [  91.0000,  294.0000, -185.5000],
        [   0.0000,  686.0000,  245.0000]])

In [10]:
dequant_tensor = scale * (quantized_tensor- zero)
dequant_tensor


tensor([[ 192.5000,  -14.0000, -206.5000],
        [  91.0000,  294.0000, -185.5000],
        [   0.0000, -210.0000,  245.0000]])

In [11]:
def linear_DQ(quantized_tensor, scale, zero):
    return scale * (quantized_tensor- zero)

In [ ]:
# ploting helper function for quantization error 


linear quantization mao [rmin , rmax] to the [qmin , qmax]

rmin = s(qmin - z)
rmax = s(qmax - z)

so for s : 

s = (rmax -rmin) / (qmax -qmin)

and for z: 

z = int(round(qmin - rmin / s) --> same data type as quantized tensor


for the prevuos example find s and z ? 


handling the edge cases for z 
what happen if tyhe zero value is out of range 

if z > qmax >> z = qmax 
if z < qmin >> z = qmin 

In [17]:
def get_scale_and_zero_values(
        r_tensor,
        dtype = torch.int8
):
    q_max , q_min = torch.iinfo(dtype).max , torch.iinfo(dtype).min 
    r_min , r_max  = r_tensor.min().item() , r_tensor.max().item()

    scale = (r_max - r_min) / (q_max - q_min) 
    z = q_min - (r_min/scale)

    if q_min <=z <= q_max :
        z = int(round(z))
    elif z < q_min : 
        z = q_min 
    elif z < q_max : 
        z = q_max

    return scale , z 
    
 

In [18]:
new_scale , new_zero = get_scale_and_zero_values(test_tensor, torch.int8)

print(new_scale)
print(new_zero)

3.578823433670343
-77


In [19]:
quantized_tensor = linear_Q(test_tensor, new_scale, new_zero)
dequant_tensor = linear_DQ (quantized_tensor, new_scale, new_zero) 

In [20]:
print(quantized_tensor)
print(dequant_tensor)

tensor([[ -23,  -81,  127],
        [ -51,    6, -128],
        [ -77,  114,   -8]], dtype=torch.int8)
tensor([[ 193.2565,  -14.3153, -186.0988],
        [  93.0494,  297.0423, -182.5200],
        [   0.0000, -232.6235,  246.9388]])


In [21]:
# test the error again

In [23]:
# make a general linear quantazation funcion:
def linear_quantizer(tensor,dtype = torch.int8):
    scale , zero_point = get_scale_and_zero_values(tensor, dtype=dtype)
    quantized_tensor = linear_Q(tensor, scale , zero_point, dtype=dtype)
    return quantized_tensor , scale , zero_point


In [24]:
r_tensor = torch.randn((4,4))
r_tensor

tensor([[ 0.7720, -0.7745,  1.0395,  0.3962],
        [-0.2456, -0.2868, -1.5821, -0.9948],
        [ 0.2450, -0.1396, -0.5027,  0.1406],
        [ 0.3538, -1.8619,  1.3330, -0.5275]])

In [25]:
quantized_tensor , scale , zero = linear_quantizer(r_tensor)

In [26]:
quantized_tensor

tensor([[  83,  -41,  104,   53],
        [   1,   -2, -105,  -58],
        [  41,   10,  -19,   32],
        [  49, -128,  127,  -21]], dtype=torch.int8)

In [27]:
zero 

21

In [28]:
scale

0.01252906649720435